# Comprehensive Analysis of Datasets with Various Metrics, Norms, and Methods

In [3]:
import sys
sys.path.append('../')

import numpy as np
from AnalysisTools import Ana
from AnalysisTools import ComputeHelpersCPU

# Initialize the analysis object
cachePath = '/home/diego/disks/ANACACHE'
comp = ComputeHelpersCPU(memory_location=cachePath, memory_verbosity=0, n_jobs=8)
analysis = Ana(showPlots=True, execution_mode=comp, cacheStoragePath=cachePath)

# Add datasets
analysis.add_dataset(label="IMR90SIM", folder="data/IMR90SIM")
analysis.add_dataset(label='IMR90OPT', folder='data/IMR90OPT')
analysis.add_dataset(label="IMR90OPT57", folder='data/IMR90OPT56')

# Process trajectories for each dataset
analysis.process_trajectories(label="IMR90SIM", filename="traj_chr_IMR90OPT_0.cndb", folder_pattern=['iteration_', [1, 20]])
analysis.process_trajectories(label="IMR90OPT", filename="traj_0.cndb", folder_pattern=['iteration_', [1, 20]])
analysis.process_trajectories(label="IMR90OPT57", filename="traj_0.cndb", folder_pattern=['iteration_', [1, 20]])

ImportError: cannot import name 'ComputeHelpers' from 'AnalysisTools.Comp_Helper_CPU' (/home/diego/Dev/OpenMiChroM-Ana/test/../AnalysisTools/Comp_Helper_CPU.py)

## Generate and Cache Distance Matrices

In [2]:
norms = ['ice', 'kr', 'log_transform', 'vc']
metrics = ['euclidean', 'pearsons', 'spearman', 'contact', 'log2_contact']
methods = ['single', 'complete', 'average', 'weighted']
#! note single/spearman/ice leads to division of 0 and all data gets set to 1 to avoid division of 0 error
#! note single/log2_contact/ice leads to division of 0 and all data divided by zero will be set to 0
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.calc_XZ("IMR90OPT", "IMR90OPT57", "IMR90SIM", metric=metric, norm=norm, method=method)

Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'single', 'euclidean', 'ice').pkl.npz
Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'complete', 'euclidean', 'ice').pkl.npz
Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'average', 'euclidean', 'ice').pkl.npz
Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'weighted', 'euclidean', 'ice').pkl.npz
Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'single', 'pearsons', 'ice').pkl.npz
Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'complete', 'pearsons', 'ice').pkl.npz
Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'average', 'pearsons', 'ice').pkl.npz
Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', '

2024-07-21 03:26:51.497959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 03:26:51.519362: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 03:26:51.526192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 03:26:51.539954: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-21 03:26:51.668647: E external/local_xla/xla/

IMR90OPT has dist shape (20, 1173, 1173)
Processing IMR90OPT57
IMR90OPT57 has dist shape (20, 1173, 1173)
Processing IMR90SIM
IMR90SIM has dist shape (100, 1183, 1183)
Flattened distance array has shape: (140, 699153)
No cached data, creating cache file /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'complete', 'spearman', 'kr').pkl
Processing IMR90OPT
IMR90OPT has dist shape (20, 1173, 1173)
Processing IMR90OPT57
IMR90OPT57 has dist shape (20, 1173, 1173)
Processing IMR90SIM
IMR90SIM has dist shape (100, 1183, 1183)
Flattened distance array has shape: (140, 699153)
No cached data, creating cache file /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'average', 'spearman', 'kr').pkl
Processing IMR90OPT
IMR90OPT has dist shape (20, 1173, 1173)
Processing IMR90OPT57
IMR90OPT57 has dist shape (20, 1173, 1173)
Processing IMR90SIM
IMR90SIM has dist shape (100, 1183, 1183)
Flattened distance array has shape: (140, 699153)
No cached data, creatin

/home/diego/Dev/OpenMiChroM-Ana/test/../AnalysisTools/Comp_Helper_CPU.py:320: RuntimeWarning: invalid value encountered in log2
  epsilon = np.finfo(float).eps
/home/diego/Dev/OpenMiChroM-Ana/test/../AnalysisTools/Comp_Helper_CPU.py:320: RuntimeWarning: invalid value encountered in log2
  epsilon = np.finfo(float).eps
/home/diego/Dev/OpenMiChroM-Ana/test/../AnalysisTools/Comp_Helper_CPU.py:320: RuntimeWarning: invalid value encountered in log2
  epsilon = np.finfo(float).eps
/home/diego/Dev/OpenMiChroM-Ana/test/../AnalysisTools/Comp_Helper_CPU.py:320: RuntimeWarning: invalid value encountered in log2
  epsilon = np.finfo(float).eps
/home/diego/Dev/OpenMiChroM-Ana/test/../AnalysisTools/Comp_Helper_CPU.py:320: RuntimeWarning: invalid value encountered in log2
  epsilon = np.finfo(float).eps
/home/diego/Dev/OpenMiChroM-Ana/test/../AnalysisTools/Comp_Helper_CPU.py:320: RuntimeWarning: invalid value encountered in log2
  epsilon = np.finfo(float).eps
/home/diego/Dev/OpenMiChroM-Ana/test/../

IMR90OPT has dist shape (20, 1173, 1173)
Processing IMR90OPT57
IMR90OPT57 has dist shape (20, 1173, 1173)
Processing IMR90SIM
IMR90SIM has dist shape (100, 1183, 1183)
Flattened distance array has shape: (140, 699153)
No cached data, creating cache file /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'complete', 'log2_contact', 'kr').pkl
Processing IMR90OPT
IMR90OPT has dist shape (20, 1173, 1173)
Processing IMR90OPT57
IMR90OPT57 has dist shape (20, 1173, 1173)
Processing IMR90SIM
IMR90SIM has dist shape (100, 1183, 1183)
Flattened distance array has shape: (140, 699153)
No cached data, creating cache file /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'average', 'log2_contact', 'kr').pkl
Processing IMR90OPT
IMR90OPT has dist shape (20, 1173, 1173)
Processing IMR90OPT57
IMR90OPT57 has dist shape (20, 1173, 1173)
Processing IMR90SIM
IMR90SIM has dist shape (100, 1183, 1183)
Flattened distance array has shape: (140, 699153)
No cached data,

KeyboardInterrupt: 

## Dimensionality Reduction Techniques

### PCA Analysis

In [4]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.pca("IMR90OPT", "IMR90OPT57", "IMR90SIM", metric=metric, n_components=-1, norm=norm, method=method)

Using cached data: /home/diego/disks/ANACACHE/cache_('IMR90OPT', 'IMR90OPT57', 'IMR90SIM', 'single', 'euclidean', 'ice').pkl.npz


TypeError: ComputeHelpers.run_reduction() takes 4 positional arguments but 5 were given

### UMAP Analysis

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.umap("IMR90OPT", "IMR90OPT56", "IMR90SIM", metric=metric, num_clusters=-1, norm=norm, method=method)

### t-SNE Analysis

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.tsne("IMR90OPT", "IMR90OPT56", "IMR90SIM", metric=metric, num_clusters=-1, norm=norm, method=method)

### MDS Analysis

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.mds("IMR90OPT", "IMR90OPT56", "IMR90SIM", metric=metric, n_components=-1, norm=norm, method=method)

### SVD Analysis

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.svd("IMR90OPT", "IMR90OPT56", "IMR90SIM", metric=metric, n_components=-1, norm=norm, method=method)

## Clustering Techniques

### K-means Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.kmeans_clustering("IMR90OPT", "IMR90OPT56", "IMR90SIM", n_clusters=5, metric=metric, norm=norm, method=method)

### DBSCAN Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.dbscan_clustering("IMR90OPT", "IMR90OPT56", "IMR90SIM", eps=0.5, min_samples=5, metric=metric, norm=norm, method=method)

### Hierarchical Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.hierarchical_clustering("IMR90OPT", "IMR90OPT56", "IMR90SIM", n_clusters=5, metric=metric, norm=norm, method=method)

### Spectral Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.spectral_clustering("IMR90OPT", "IMR90OPT56", "IMR90SIM", num_clusters=-1, metric=metric, norm=norm, method=method)

### OPTICS Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.optics_clustering("IMR90OPT", "IMR90OPT56", "IMR90SIM", min_samples=5, xi=0.05, min_cluster_size=0.05, metric=metric, norm=norm, method=method)